In [5]:
import tensorflow as tf
import numpy as np
import keras
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Conv2DTranspose
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

In [3]:
## Generator Network ##
gen_model = Sequential()
gen_model.add(Dense())

In [ ]:
# Discriminator
self.D = Sequential()
depth = 64
dropout = 0.4
# In: 28 x 28 x 1, depth = 1
# Out: 14 x 14 x 1, depth=64
input_shape = (self.img_rows, self.img_cols, self.channel)
self.D.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape,\
padding='same', activation=LeakyReLU(alpha=0.2)))
self.D.add(Dropout(dropout))
self.D.add(Conv2D(depth*2, 5, strides=2, padding='same',\
activation=LeakyReLU(alpha=0.2)))
self.D.add(Dropout(dropout))
self.D.add(Conv2D(depth*4, 5, strides=2, padding='same',\
activation=LeakyReLU(alpha=0.2)))
self.D.add(Dropout(dropout))
self.D.add(Conv2D(depth*8, 5, strides=1, padding='same',\
activation=LeakyReLU(alpha=0.2)))
self.D.add(Dropout(dropout))
# Out: 1-dim probability
self.D.add(Flatten())
self.D.add(Dense(1))
self.D.add(Activation('sigmoid'))
self.D.summary()

In [ ]:
self.G = Sequential()
dropout = 0.4
depth = 64+64+64+64 #256
dim = 7
# In: 100
# Out: dim x dim x depth
self.G.add(Dense(dim*dim*depth, input_dim=100))
self.G.add(BatchNormalization(momentum=0.9))
self.G.add(Activation('relu'))
self.G.add(Reshape((dim, dim, depth)))
self.G.add(Dropout(dropout))
# In: dim x dim x depth
# Out: 2*dim x 2*dim x depth/2
self.G.add(UpSampling2D())
self.G.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
self.G.add(BatchNormalization(momentum=0.9))
self.G.add(Activation('relu'))
self.G.add(UpSampling2D())
self.G.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
self.G.add(BatchNormalization(momentum=0.9))
self.G.add(Activation('relu'))
self.G.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
self.G.add(BatchNormalization(momentum=0.9))
self.G.add(Activation('relu'))
# Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
self.G.add(Conv2DTranspose(1, 5, padding='same'))
self.G.add(Activation('sigmoid'))
self.G.summary()
return self.G

In [24]:
## Test on MNIST data
## Input = 100
## Output = dim * dim * depth
dim = 7
depth = 256

model = Sequential()
model.add(Dense(dim * dim * depth, input_dim=100))
model.add(LeakyReLU())
model.add(Reshape((dim, dim, depth)))
model.add(Dropout(0.2))
print(model.output)
model.add(UpSampling2D())
print(model.output)
## Conv2DTranspose = inverse of a convolutional layer
model.add(Conv2DTranspose(filters=128, kernel_size=(4, 4), strides=1, padding='same'))
print(model.output)
model.add(LeakyReLU())
model.add(UpSampling2D())
print(model.output)
model.add(Conv2DTranspose(filters=64, kernel_size=(4, 4), strides=1, padding='same'))
print (model.output)
model.add(LeakyReLU())
model.add(Conv2DTranspose(filters=32, kernel_size=(4, 4), strides=1, padding='same'))
model.add(LeakyReLU())
model.add(Conv2DTranspose(filters=1, kernel_size=(4, 4), strides=1, padding='same'))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Tensor("dropout_20/cond/Merge:0", shape=(?, 7, 7, 256), dtype=float32)
Tensor("up_sampling2d_28/ResizeNearestNeighbor:0", shape=(?, 14, 14, 256), dtype=float32)
Tensor("conv2d_transpose_31/BiasAdd:0", shape=(?, ?, ?, 128), dtype=float32)
Tensor("up_sampling2d_29/ResizeNearestNeighbor:0", shape=(?, 28, 28, 128), dtype=float32)
Tensor("conv2d_transpose_32/BiasAdd:0", shape=(?, ?, ?, 64), dtype=float32)


In [43]:
test = np.random.rand(100)
test = [test]
test = np.asarray(test)
# test.shape

ValueError: Error when checking target: expected activation_3 to have 4 dimensions, but got array with shape (2, 1)

In [26]:
model = Sequential()
model.add(UpSampling2D(size=(2, 2), input_shape=(1, 16, 16)))
model.build()
# keras_param_test(model, 0, 3)

In [27]:
model.output

<tf.Tensor 'up_sampling2d_3/ResizeNearestNeighbor:0' shape=(?, 2, 32, 16) dtype=float32>